# Introduzione

L'applicazione permette agli utenti di monitorare i valori dei parametri di stampaggio.

Gli utenti selezionano i codici **part number**, **tag stampo**, **pressa** e **ordine**, quindi impostano i limiti per ogni parametro di stampaggio corrispondente alla combinazione di codici scelta. I dati relativi a ciascun parametro vengono recuperati e graficati all'interno della loro rispettiva carta di controllo.

La documentazione descriverà i vari componenti del progetto (dati, database, backend, frontend), spiegando le loro funzionalità e come interagiscono.

# Esecuzione

Per testare l'applicazione:

1. **Eseguire il file `app_MTD.py`**:
   - Avere un ambiente di sviluppo configurato correttamente.
   - Aprire la cartella contenente il file `app_MTD.py` all'interno dell'ambiente di sviluppo.
   - Avviare il file.

2. **Accedere all'applicazione tramite il browser**:
   - Una volta che il file è in esecuzione, sul terminale apparirà l'indirizzo IP locale.
   - Utilizzare il comando `Ctrl + Click` sull'indirizzo IP visualizzato nel terminale per aprirlo direttamente nel browser.
   - In alternativa, aprire il browser e digitare `localhost:5000` (o l'indirizzo che appare nel terminale) nella barra di ricerca del browser.

3. **Accesso all'applicazione**:
   - Una volta aperto l'indirizzo nel browser, si verrà reindirizzati alla prima schermata dell'applicazione, pronta per interagire con i dati e le funzionalità.


# Dati

I dati utilizzati nell'applicazione provengono da un file Excel generato a partire dall'E-DHR (Electronic Device History Record). I fogli di lavoro contenenti i dati di nostro interesse sono stati esportati in formato CSV, manipolati e spostati nella cartella `Datasets`.


## Manipolazione dei Dati


La manipolazione dei dati avviene tramite l'esecuzione dello script `pulizia_csv.py`. Questo script esegue le seguenti operazioni:

- **Caricamento del CSV**: Il file CSV viene caricato utilizzando la libreria `pandas` con il corretto delimitatore e l'encoding `latin1` per gestire i caratteri speciali.
   
- **Funzione di separazione dei valori**: Una funzione `separa_valore_unità` viene utilizzata per separare i valori numerici dalle unità di misura o dal simbolo di percentuale, se presenti. Se il valore è accompagnato dal simbolo `%`, viene trattato come una percentuale.

- **Pulizia e trasformazione**: La funzione applicata alla colonna 'Valore' del CSV crea due nuove colonne: una per il valore numerico e l'altra per l'unità di misura.

- **Salvataggio del file modificato**: Dopo aver effettuato le modifiche, il DataFrame viene salvato nuovamente nel file CSV, sovrascrivendo quello esistente.


# Database

Il database dell'applicazione viene creato e popolato attraverso l'esecuzione dello script `DbMaker.py`. Il file risultante, chiamato `MTD.db`, viene successivamente spostato nella cartella `DatabaseSql`.


## Creazione e Popolamento del Database


Lo script esegue le seguenti operazioni:

- **Connessione al Database**: Viene creato un database SQLite chiamato `MTD.db`, connesso tramite la libreria `sqlite3`.

- **Creazione delle Tabelle**: Vengono create quattro tabelle nel database:
   - **Ordini**: Contiene i dati relativi agli ordini.
   - **Parametri**: Contiene i parametri legati agli ordini.
   - **BOM**: Contiene la distinta base per ciascun ordine.
   - **Limiti**: Contiene i limiti dei parametri di stampaggio.

- **Inserimento dei Dati**: I dati vengono letti dai file CSV (dalla cartella `Datasets`) e inseriti nelle rispettive tabelle del database in blocchi di 1000 righe per volta, utilizzando la libreria `pandas`.

- **Gestione degli Errori**: Durante l'inserimento, vengono gestiti errori di integrità (come chiavi duplicate) e il processo prosegue senza interruzioni.

- **Chiusura della Connessione**: Una volta completato l'inserimento dei dati, la connessione al database viene chiusa per salvare le modifiche.


# Backend

Il backend dell'applicazione (`app_MTD.py`) è stato sviluppato in Python ed è costruito utilizzando il framework Flask, che gestisce le richieste HTTP e le interazioni con il database. Il codice del backend è organizzato in diverse funzioni, ciascuna con una responsabilità specifica.


## Python


### Funzione: `get_data_from_db(query, params=None)`

La funzione `get_data_from_db` esegue una query SQL sul database e restituisce i risultati della query.

- **Scopo**: Eseguire una query sul database e recuperare i risultati.
- **Parametri**:
  - `query`: La query SQL da eseguire (stringa).
  - `params`: Parametri opzionali da passare alla query (lista o tupla).
- **Output**: Una lista di tuple contenente i risultati della query eseguita.

### Funzione: `get_limiti_combinazione(pressa, tag_stampo, part_number, ordine)`

La funzione `get_limiti_combinazione` recupera i limiti associati ad ogni parametro corrispondente ad una combinazione specifica di pressa, tag stampo, part number e ordine.

- **Scopo**: Ottenere i limiti associati ai parametri corrispondenti alla combinazione di pressa, tag stampo, part number e ordine scelta dall'utente.
- **Parametri**:
  - `pressa`: Il nome della pressa (stringa).
  - `tag_stampo`: Il tag stampo (stringa).
  - `part_number`: Il part number (stringa).
  - `ordine`: Il numero dell'ordine (stringa).
- **Output**: Una lista di tuple contenente i limiti (Parametro, Limite_inf, Limite_sup) per la combinazione specificata.

### Funzione: `get_ordini_filtrati()`

La funzione `get_ordini_filtrati` gestisce una richiesta API per ottenere gli ordini filtrati in base ai parametri passati tramite la query string.

- **Scopo**: Recuperare gli ordini filtrati in base ai parametri di pressa, tag stampo e part number.
- **Parametri**:
  - `pressa`: Il parametro per filtrare gli ordini in base al codice della pressa (stringa).
  - `tag_stampo`: Il parametro per filtrare gli ordini in base al tag stampo (stringa).
  - `part_number`: Il parametro per filtrare gli ordini in base al part number (stringa).
- **Output**: Una lista di ordini distinti (id e nome) restituita in formato JSON.

### Funzione: `get_tag_stampo()`

La funzione `get_tag_stampo` gestisce una richiesta API per ottenere i tag stampo filtrati in base ai parametri passati tramite la query string.

- **Scopo**: Recuperare i tag stampo univoci filtrati in base ai parametri di pressa e part number.
- **Parametri**:
  - `pressa`: Il parametro per filtrare i tag stampo in base al codice della pressa (stringa) (opzionale).
  - `part_number`: Il parametro per filtrare i tag stampo in base al part number (stringa) (opzionale).
- **Output**: Una lista di tag stampo univoci (id e nome) restituita in formato JSON.

### Funzione: `get_part_number()`

La funzione `get_part_number` gestisce una richiesta API per ottenere i part number filtrati in base ai parametri passati tramite la query string.

- **Scopo**: Recuperare i part number univoci filtrati in base ai parametri di pressa e tag stampo.
- **Parametri**:
  - `pressa`: Il parametro per filtrare i part number in base al codice della pressa (stringa) (opzionale).
  - `tag_stampo`: Il parametro per filtrare i part number in base al tag stampo (stringa) (opzionale).
- **Output**: Una lista di part number univoci (id e nome) restituita in formato JSON.

### Funzione: `get_pressa()`

La funzione `get_pressa` gestisce una richiesta API per ottenere le presse filtrate in base ai parametri passati tramite la query string.

- **Scopo**: Recuperare le presse univoci filtrate in base ai parametri di tag stampo e part number.
- **Parametri**:
  - `tag_stampo`: Il parametro per filtrare le presse in base al tag stampo (stringa) (opzionale).
  - `part_number`: Il parametro per filtrare le presse in base al part number (stringa) (opzionale).
- **Output**: Una lista di presse univoci (id e nome) restituita in formato JSON.

### Funzione: `index()`

La funzione `index` gestisce la richiesta GET per la pagina principale, recuperando i dati necessari per popolarsi dinamicamente nella vista.

- **Scopo**: Recuperare i codici pressa, i tag stampo e i part number, ordinati in ordine decrescente, e renderizzare la pagina principale con i dati.
- **Parametri**: Nessun parametro in ingresso (la funzione è legata alla visualizzazione della pagina principale).
- **Output**: Restituisce una pagina HTML renderizzata (`nuovo_selezione_codici.html`), passando i dati recuperati per i codici pressa, i tag stampo e i part number.

### Funzione: `inserimento_limiti()`

La funzione `inserimento_limiti` gestisce la visualizzazione e l'inserimento dei limiti per un dato ordine, pressa, tag stampo e part number. Può gestire sia il caso di ordini multipli che di un singolo ordine.

- **Scopo**: Recuperare i parametri e i limiti per un ordine specificato (o per tutti gli ordini) e renderizzare il template `inserimento_limiti.html` con i dati necessari per l'inserimento dei limiti.
- **Parametri**:
  - `pressa`: Il codice della pressa.
  - `tag_stampo`: Il tag stampo.
  - `part_number`: Il part number.
  - `ordine`: L'ordine, che può essere "Tutti" o un valore specifico.
- **Output**: Restituisce il template `inserimento_limiti.html`, passando i dati relativi a ordini, parametri e limiti.

### Funzione: `submit_limiti()`

La funzione `submit_limiti` gestisce l'inserimento o l'aggiornamento dei limiti per un ordine specificato (pressa, tag stampo, part number e ordine). Si occupa di ricevere i dati da un form, verificare la loro validità e poi aggiornarli o inserirli nel database.

- **Scopo**: Inserire o aggiornare i limiti di stampaggio per i parametri specificati, associandoli ai valori di pressa, tag stampo, part number e ordine. Alla fine, redirige l'utente alla pagina dei grafici con i limiti appena salvati.
- **Parametri**:
  - `pressa`: Il codice della pressa.
  - `tag_stampo`: Il tag stampo.
  - `part_number`: Il part number.
  - `ordine`: L'ordine di riferimento.
  - `parametri`: La lista di parametri per cui inserire o aggiornare i limiti.
  - `limite_inf`: I limiti inferiori dei parametri.
  - `limite_sup`: I limiti superiori dei parametri.
- **Output**: Reindirizza l'utente alla pagina dei grafici (`/grafici`), passando i limiti e altre informazioni relative all'ordine, pressa, tag stampo e part number.

### Funzione: `grafici()`

La funzione `grafici()` visualizza i dati dei parametri per un ordine, pressa, tag stampo e part number, generando grafici interattivi che mostrano i valori dei parametri nel tempo, con l'evidenziazione dei valori che rientrano o meno nei limiti definiti.

- **Scopo**: Recuperare i dati relativi a parametri, limiti e ordini da visualizzare in grafici interattivi. Mostra i valori che sono all'interno e fuori dai limiti definiti.
- **Parametri**:
  - `pressa`: Il codice della pressa.
  - `tag_stampo`: Il tag stampo.
  - `part_number`: Il part number.
  - `ordine`: L'ordine o gli ordini selezionati. Può essere una lista o un singolo valore.
  - `limite_inf`: Limiti inferiori dei parametri.
  - `limite_sup`: Limiti superiori dei parametri.
- **Output**: Restituisce il template `grafici.html`, contenente i grafici generati con i dati relativi ai parametri, i limiti inferiori e superiori, e altre informazioni come l'ordine, l'unità di misura e il lotto.

# Frontend

Il frontend dell'applicativo comprende diversi componenti, quali l'HTML (cartella `templates`), il CSS (cartella `static`) e il JavaScript (cartella `static` → cartella `js`), necessari per la visualizzazione e l'interazione dell'utente con i dati. Ogni componente ha un ruolo specifico per garantire un'interfaccia utente chiara e funzionale.


## HTML


### File: `nuovo_selezione_codici.html`

Il file `nuovo_selezione_codici.html` permette all'utente di selezionare i parametri relativi a **Part Number**, **Tag Stampo**, **Pressa** e **Ordine** tramite menu a discesa. I dati vengono inviati tramite un form per l'inserimento dei limiti.

#### Componenti principali:
- **Selezioni a discesa**: I `select` vengono popolati dinamicamente con dati dal backend tramite Jinja.
- **Interfaccia utente**: Utilizza **Bootstrap** per il layout e **Select2** per migliorare l'usabilità delle selezioni.
- **Funzionalità JavaScript**: Inizializzazione di Select2 e script personalizzati per la gestione dei filtri nelle selezioni.
- **Invio del form**: I dati selezionati vengono inviati tramite POST alla route `/inserimento_limiti`.

### File: `inserimento_limiti.html`

Il file `inserimento_limiti.html` consente di inserire i limiti inferiori e superiori per vari parametri associati a un ordine, pressa, tag stampo e part number specifici.

#### Componenti principali:
- **Selezione dei parametri**: Per ogni parametro, viene mostrato un campo per inserire i limiti inferiori e superiori.
- **Form nascosto**: I dati relativi a pressa, tag stampo, part number e ordine sono inviati come input nascosti per mantenere il contesto.
- **Interfaccia utente**: Utilizza **Bootstrap** per il layout delle card e dei campi di input.
- **Invio del form**: I limiti inseriti vengono inviati tramite un form POST alla route `/submit_limiti` per il salvataggio.
- **Navigazione**: Un pulsante permette di tornare indietro alla schermata precedente.

### File: `grafici.html`

Il file `grafici.html` visualizza i grafici per i parametri selezionati in base ai limiti inseriti, mostrando i dati per un ordine, pressa, tag stampo e part number specifici.

#### Componenti principali:
- **Visualizzazione dei grafici**: I grafici generati vengono inseriti dinamicamente nella pagina utilizzando un ciclo `for` per ciascun grafico.
- **Dettagli dei codici**: Mostra i codici selezionati (pressa, tag stampo, part number, ordine) come sottotitolo.
- **Navigazione**: Due pulsanti di navigazione sono presenti:
  - Uno per tornare alla schermata precedente.
  - Uno per tornare alla schermata di selezione dei codici.


## CSS


### File: `styles_boots.css`

Il file `styles_boots.css` personalizza l'aspetto dell'applicazione, costruendo sopra il framework Bootstrap e adattandolo a specifici requisiti.

#### Componenti principali:

- **Logo**: Larghezza fissa di 200px.
- **Titoli**: Stile personalizzato per i titoli e sottotitoli.
- **Form**: Disposizione a colonna dei campi e centratura delle card.
- **Input**: Input a larghezza piena con stile al focus.
- **Hover sulle Card**: Effetto di ingrandimento e ombra.
- **Bottoni**: Pulsanti flottanti con animazione e stile secondario.
- **Responsive**: Layout adattivo per dispositivi mobili.


## JavaScript


### File: `script_select.js`

Il file `script_select.js` gestisce l'inizializzazione della libreria Select2 per i menu a tendina (dropdown) presenti nelle varie schermate dell'applicazione.

#### Funzionalità principale:

- **Inizializzazione di Select2**: Attiva Select2 per i seguenti dropdown:
  - `TAG_stampo_input`: Consente di selezionare il "Tag Stampo" con un placeholder e l'opzione di svuotare la selezione.
  - `codice_pressa_input`: Consente di selezionare la "Pressa" con un placeholder e l'opzione di svuotare la selezione.
  - `ordine_input`: Consente di selezionare il "Part Number" con un placeholder e l'opzione di svuotare la selezione.
  - `part_number_input`: Consente di selezionare l'"Ordine" con un placeholder e l'opzione di svuotare la selezione.

### File: `dropdown.js`

Il file `dropdown.js` gestisce la logica di interazione tra i vari menu a tendina (dropdown) dell'applicazione, aggiornandoli dinamicamente in base alle selezioni effettuate.

#### Funzionalità principali:

- **Inizializzazione Select2**: Attiva il plugin Select2 sui seguenti dropdown:
  - `codice_pressa_input`: Consente di selezionare una "Pressa".
  - `TAG_stampo_input`: Consente di selezionare un "Tag Stampo".
  - `part_number_input`: Consente di selezionare un "Part Number".

- **Gestione filtri dinamici**: Quando una selezione cambia in uno dei dropdown, la funzione `updateDropdowns` viene chiamata per aggiornare gli altri dropdown in base ai filtri precedentemente selezionati. Viene verificato che i filtri non siano cambiati, evitando chiamate API inutili.

- **Chiamate API per aggiornamento dei dati**: I dropdown vengono aggiornati con nuove opzioni tramite chiamate AJAX alle API (`/api/Tag_stampo`, `/api/part_number`, `/api/pressa`). I dati vengono ricevuti, processati, e le opzioni del menu a tendina vengono aggiornate di conseguenza.

- **Memorizzazione stato filtri**: L'ultimo stato dei filtri selezionati viene salvato in `lastFilters`, in modo da evitare chiamate duplicate se i filtri non sono cambiati.

### File: `script2.js`

Il file `script2.js` gestisce il caricamento dinamico del dropdown "Ordine" in base ai filtri selezionati (Pressa, Tag Stampo, Part Number). Utilizza jQuery e la libreria Select2 per la gestione dei dropdown, oltre ad AJAX per il caricamento dei dati dal server.

#### Componenti principali:
- **Inizializzazione Select2**: Applica `select2` sui dropdown per una migliore esperienza utente.
  - Ogni campo (Pressa, Tag Stampo, Part Number, Ordine) è inizializzato con un placeholder personalizzato.
  
- **Funzione `updateOrdine`**: Quando l'utente seleziona i filtri (Pressa, Tag Stampo, Part Number), questa funzione gestisce il caricamento dei dati per il dropdown "Ordine".
  - **Caricamento**: Quando i filtri sono selezionati, viene mostrato un messaggio di caricamento con un'animazione dei punti ("Caricamento.", "Caricamento..", "Caricamento...").
  - **AJAX**: Viene effettuata una richiesta AJAX per ottenere i dati relativi agli ordini dal server, aggiornando dinamicamente le opzioni nel dropdown "Ordine".
  - Se non ci sono ordini corrispondenti, viene visualizzato un messaggio di "Nessun ordine corrispondente".
  
- **Gestione degli errori**: Se si verifica un errore durante il caricamento, l'animazione di caricamento viene fermata e il dropdown "Ordine" viene abilitato.

#### Funzionalità:
- **Animazione di caricamento**: Durante il caricamento dei dati, viene mostrato un indicatore di caricamento con animazione dei punti.
- **Abilitazione e disabilitazione dinamica dei dropdown**: Il dropdown "Ordine" viene disabilitato durante il caricamento e riabilitato quando i dati sono stati caricati con successo.
- **Eventi di cambio**: Ogni cambiamento nei filtri (Pressa, Tag Stampo, Part Number) attiva la funzione di aggiornamento del dropdown "Ordine".
